<a href="https://colab.research.google.com/github/ChoiJK/JK_Mini_ImgNet_Classification/blob/master/Mini_ImageNet_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import random

TensorFlow 2.x selected.


### DataSet Load

In [2]:
#load mini-imagenet dataset file from www.youplayai.com
train= np.load('train.npy')
val = np.load('val.npy')
#test = np.load('test.npy')
train_label= np.load('train_label.npy')
val_label = np.load('val_label.npy')
#test_label = np.load('test_label.npy')

FileNotFoundError: ignored